In [1]:
!pip install transformers torch scikit-learn

In [13]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

dataset_path = 'dataset.csv'
df = pd.read_csv(dataset_path)

print("DataFrame Head:\n", df.head())

print("Columns:\n", df.columns)

print("Missing Values:\n", df.isnull().sum())

# Ensure 'Content_1' and 'Genre' columns are present
if 'Content_1' not in df.columns or 'Genre' not in df.columns:
    raise ValueError("Required columns 'Content_1' or 'Genre' are missing in the dataframe.")

# Drop rows with missing 'Content_1' or 'Genre'
df = df.dropna(subset=['Content_1', 'Genre'])

# Reset index to ensure proper indexing
df = df.reset_index(drop=True)

# Display the first few rows after cleaning
print("Cleaned DataFrame Head:\n", df.head())

# Encode labels
label_encoder = LabelEncoder()
df['Genre_encoded'] = label_encoder.fit_transform(df['Genre'])

# Display encoded labels
print("Encoded Labels:\n", df['Genre_encoded'].head())

# Split the dataset into train, validation, and test sets
train_df, val_df, test_df = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

# Check the splits
print(f"Training Set Size: {len(train_df)}")
print(f"Validation Set Size: {len(val_df)}")
print(f"Test Set Size: {len(test_df)}")

# Initialize ParsBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')

# Custom dataset class for Persian summaries
class PersianMovieGenreDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts.reset_index(drop=True)  # Ensure proper indexing
        self.labels = labels.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        try:
            text = self.texts.iloc[idx]
            label = self.labels.iloc[idx]
        except IndexError as e:
            print(f"IndexError: {e}, idx: {idx}, len(texts): {len(self.texts)}, len(labels): {len(self.labels)}")
            raise
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create datasets
train_dataset = PersianMovieGenreDataset(train_df['Content_1'], train_df['Genre_encoded'], tokenizer)
val_dataset = PersianMovieGenreDataset(val_df['Content_1'], val_df['Genre_encoded'], tokenizer)
test_dataset = PersianMovieGenreDataset(test_df['Content_1'], test_df['Genre_encoded'], tokenizer)

# Create dataloaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Load the ParsBERT model
model = AutoModelForSequenceClassification.from_pretrained('HooshvareLab/bert-fa-base-uncased', num_labels=len(label_encoder.classes_))
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Evaluate the model on the test set
results = trainer.evaluate(test_dataset)
print(results)

# Save the model and tokenizer
model.save_pretrained('./parsbert_movie_genre_classifier')
tokenizer.save_pretrained('./parsbert_movie_genre_classifier')


DataFrame Head:
                                                 Link            EN_title  \
0  https://www.imvbox.com/watch-persian-movie-ira...   Local Anaesthetic   
1  https://www.imvbox.com/watch-persian-movie-ira...         Disturbance   
2  https://www.imvbox.com/watch-persian-movie-ira...           Highlight   
3  https://www.imvbox.com/watch-persian-movie-ira...               Gilda   
4  https://www.imvbox.com/watch-persian-movie-ira...  Atmosphere Station   

     PENGLISH_title   PERSIAN_title  \
0  Bi Hessie Mozeie    بی‌حسی موضعی   
1         Ashoftegi        آشفته گی   
2           Haylayt         هایلایت   
3            Geelda           گیلدا   
4  Istgahe Atmosfer  ایستگاه اتمسفر   

                                           Content_1  \
0  جلال‌، دانشجوی سابق رشته فلسفه، متوجه می‌شود خ...   
1  «آشفته‌گی» رئالیستی و اجتماعی نیست. یک فیلم اس...   
2  یک تصادف اتومبیل آدم‌هایی را در تقابل با هم قر...   
3  گیلدا ماجرای زنی به نام «گیلدا» را روایت می کن...   
4  این فیلم

c:\Users\Fateme\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Fateme\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/324 [00:00<?, ?it/s]

{'loss': 2.7875, 'grad_norm': 13.324082374572754, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.09}
{'loss': 2.8199, 'grad_norm': 12.376668930053711, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.19}
{'loss': 2.5429, 'grad_norm': 10.730721473693848, 'learning_rate': 3e-06, 'epoch': 0.28}
{'loss': 2.3365, 'grad_norm': 12.58212661743164, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.37}
{'loss': 2.1922, 'grad_norm': 10.538143157958984, 'learning_rate': 5e-06, 'epoch': 0.46}
{'loss': 1.9286, 'grad_norm': 11.076147079467773, 'learning_rate': 6e-06, 'epoch': 0.56}
{'loss': 1.7189, 'grad_norm': 7.290389060974121, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.65}
{'loss': 1.6896, 'grad_norm': 7.394067764282227, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.74}
{'loss': 1.6942, 'grad_norm': 6.355474472045898, 'learning_rate': 9e-06, 'epoch': 0.83}
{'loss': 1.4974, 'grad_norm': 5.067628383636475, 'learning_rate': 1e-05, 'epoch': 0.93}


  0%|          | 0/14 [00:00<?, ?it/s]

...

  0%|          | 0/14 [00:00<?, ?it/s]

{'eval_loss': 1.2769464254379272, 'eval_runtime': 78.2225, 'eval_samples_per_second': 1.381, 'eval_steps_per_second': 0.179, 'epoch': 3.0}
{'train_runtime': 5050.6151, 'train_samples_per_second': 0.511, 'train_steps_per_second': 0.064, 'train_loss': 1.6508636504043768, 'epoch': 3.0}


  0%|          | 0/14 [00:00<?, ?it/s]

{'eval_loss': 1.2469316720962524, 'eval_runtime': 78.4615, 'eval_samples_per_second': 1.376, 'eval_steps_per_second': 0.178, 'epoch': 3.0}


('./parsbert_movie_genre_classifier\\tokenizer_config.json',
 './parsbert_movie_genre_classifier\\special_tokens_map.json',
 './parsbert_movie_genre_classifier\\vocab.txt',
 './parsbert_movie_genre_classifier\\added_tokens.json',
 './parsbert_movie_genre_classifier\\tokenizer.json')

In [30]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

predictions = trainer.predict(test_dataset)
y_true = test_df['Genre_encoded'].to_numpy()
y_pred = np.argmax(predictions.predictions, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
conf_matrix = confusion_matrix(y_true, y_pred)

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'F1 Score (Macro): {f1_macro}')
print(f'F1 Score (Micro): {f1_micro}')
print(f'Precision (Macro): {precision}')
print(f'Recall (Macro): {recall}')
print('Confusion Matrix:')
print(conf_matrix)

  0%|          | 0/14 [00:00<?, ?it/s]

Accuracy: 0.6388888888888888
F1 Score (Macro): 0.212359900373599
F1 Score (Micro): 0.6388888888888888
Precision (Macro): 0.2773109243697479
Recall (Macro): 0.2091723093371347
Confusion Matrix:
[[ 2  0  1  0  5  0  0  0]
 [ 0  0  0  0  5  0  0  0]
 [ 0  0 12  0 11  0  0  0]
 [ 0  0  2  0  5  0  0  0]
 [ 0  0  6  0 55  0  0  0]
 [ 0  0  0  0  2  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  1  0  0  0]]


c:\Users\Fateme\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [39]:
# 20 Persian text examples with their true genres
examples = [
    ("یک دانشجوی فلسفه متوجه ازدواج پنهانی خواهرش با یک قمارباز می‌شود.", "Drama"),
    ("یک زوج در یک شهر کوچک با مشکلات مالی دست و پنجه نرم می‌کنند.", "Drama"),
    ("یک مرد تنها در تلاش برای بقا در یک جهان پس از آخرالزمان است.", "Adventure"),
    ("یک دختر جوان به دنبال رویاهای خود به یک شهر بزرگ می‌رود.", "Romance"),
    ("یک زن کارآفرین با چالش‌های ایجاد یک شرکت نوپا مواجه می‌شود.", "Drama"),
    ("یک سرباز کهنه‌کار به خانه بازمی‌گردد و با گذشته‌اش مواجه می‌شود.", "Drama"),
    ("یک کودک با نیروهای فوق‌العاده تلاش می‌کند جهان را نجات دهد.", "Fantasy"),
    ("یک خانواده در تلاش برای بقا در یک دنیای زامبی‌ها هستند.", "Horror"),
    ("یک هنرمند جوان تلاش می‌کند در دنیای هنر معروف شود.", "Drama"),
    ("یک زوج عاشق با چالش‌های خانوادگی مواجه می‌شوند.", "Romance"),
    ("یک گروه موسیقی جوان تلاش می‌کنند در صنعت موسیقی موفق شوند.", "Drama"),
    ("یک وکیل تلاش می‌کند یک پرونده جنایی بزرگ را حل کند.", "Crime"),
    ("یک نویسنده معروف با بحران خلاقیت مواجه می‌شود.", "Drama"),
]

# Define the prediction function
def predict_genre(text, model, tokenizer):
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    logits = outputs.logits
    predicted_class_idx = logits.argmax(axis=1).item()
    predicted_label = label_encoder.inverse_transform([predicted_class_idx])[0]
    
    return predicted_label

# Predict genres for the examples
true_labels = [genre for _, genre in examples]
predicted_labels = [predict_genre(text, model, tokenizer) for text, _ in examples]

# Evaluate accuracy
correct_predictions = sum([true == pred for true, pred in zip(true_labels, predicted_labels)])
accuracy = correct_predictions / len(examples)

print(f'Predicted genres: {predicted_labels}')
print(f'True genres: {true_labels}')
print(f'Number of correct predictions: {correct_predictions} out of {len(examples)}')
print(f'Accuracy: {accuracy:.2f}')


Predicted genres: ['Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'Comedy', 'Drama']
True genres: ['Drama', 'Drama', 'Adventure', 'Romance', 'Drama', 'Drama', 'Fantasy', 'Horror', 'Drama', 'Romance', 'Drama', 'Crime', 'Drama']
Number of correct predictions: 7 out of 13
Accuracy: 0.54


In [34]:
# 30 Persian text examples with their true genres
examples = [
    ("سارا در تلاش است تا با مدیریت بحران‌های خانوادگی و مشکلات شخصی‌اش زندگی بهتری برای فرزندانش فراهم کند.", "Drama"),
    ("کامران تصمیم می‌گیرد که پس از بازنشستگی به شهر زادگاهش بازگردد و خانه کودکی‌اش را بازسازی کند.", "Drama"),
    ("یک کمدین جوان در تلاش است تا با شوخی‌های نوین و خلاقانه خود مخاطبان بیشتری جذب کند.", "Comedy"),
    ("یک معلم مدرسه روستایی تلاش می‌کند تا دانش‌آموزانش را به یادگیری تشویق کند و آینده بهتری برای آنها رقم بزند.", "Drama"),
    ("مریم و علی تصمیم می‌گیرند که در یک سفر ماجراجویانه به کوه‌های شمال کشور بروند و در این سفر با اتفاقات خنده‌داری مواجه می‌شوند.", "Comedy"),
    ("یک نویسنده تلاش می‌کند تا از داستان‌های واقعی زندگی مردم یک کتاب الهام‌بخش بنویسد.", "Drama"),
    ("یک گروه موسیقی تصمیم می‌گیرند تا در یک فستیوال بزرگ شرکت کنند و در این راه با مشکلات و اتفاقات خنده‌داری روبرو می‌شوند.", "Comedy"),
    ("یک بازیگر مشهور با چالش‌های شهرت و فشارهای اجتماعی روبرو می‌شود و تلاش می‌کند تا زندگی شخصی‌اش را مدیریت کند.", "Drama"),
    ("یک پسر نوجوان که به تازگی وارد دبیرستان شده است با مشکلات جدید و چالش‌های بلوغ مواجه می‌شود.", "Drama"),
    ("یک زن و شوهر تصمیم می‌گیرند تا کسب و کار خانوادگی خود را گسترش دهند و در این راه با مشکلات خنده‌داری روبرو می‌شوند.", "Comedy"),
    ("یک زن جوان تصمیم می‌گیرد که پس از شکست عشقی بزرگ به سفر برود و در این سفر با خودآگاهی بیشتری روبرو شود.", "Drama"),
    ("یک پدر تنها تلاش می‌کند تا برای دختر کوچکش یک جشن تولد فراموش‌نشدنی برگزار کند.", "Drama"),
    ("یک کارآگاه خصوصی در تلاش است تا پرونده پیچیده یک قتل را حل کند.", "Drama"),
    ("یک استاد دانشگاه تلاش می‌کند تا با تحقیقاتی جدید و خلاقانه در حوزه علمی خود پیشرفت کند.", "Drama"),
    ("یک زن جوان در تلاش است تا یک شرکت استارتاپی موفق راه‌اندازی کند و با مشکلات زیادی مواجه می‌شود.", "Drama"),
    ("یک گروه از دوستان تصمیم می‌گیرند که تعطیلات خود را در یک ویلا در شمال کشور بگذرانند و با ماجراهای خنده‌داری مواجه می‌شوند.", "Comedy"),
    ("یک هنرمند جوان در تلاش است تا اولین نمایشگاه هنری خود را با موفقیت برگزار کند.", "Drama"),
    ("یک مربی فوتبال تلاش می‌کند تا تیم جوانان محلی را به قهرمانی برساند.", "Drama"),
    ("یک زوج مسن تصمیم می‌گیرند تا باقی‌مانده زندگی خود را به سفرهای مختلف بگذرانند و در هر سفر با اتفاقات خنده‌داری مواجه می‌شوند.", "Comedy"),
    ("یک مخترع جوان در تلاش است تا ایده‌های نوآورانه خود را به واقعیت تبدیل کند.", "Drama"),
    ("یک دختر جوان که به تازگی از دانشگاه فارغ‌التحصیل شده است، تلاش می‌کند تا شغلی مناسب پیدا کند.", "Drama"),
    ("یک نویسنده در تلاش است تا با الهام از زندگی خود یک رمان پرفروش بنویسد.", "Drama"),
    ("یک گروه از دوستان تصمیم می‌گیرند که یک رستوران کوچک افتتاح کنند و در این راه با چالش‌های خنده‌داری روبرو می‌شوند.", "Comedy"),
    ("یک مادر جوان تلاش می‌کند تا همزمان با مدیریت خانه و شغل، از کودکانش مراقبت کند.", "Drama"),
    ("یک معلم موسیقی در تلاش است تا استعدادهای نهفته دانش‌آموزانش را کشف و پرورش دهد.", "Drama"),
    ("یک زن تصمیم می‌گیرد تا پس از جدایی از همسرش، زندگی جدیدی را آغاز کند.", "Drama"),
    ("یک گروه تئاتر آماتور در تلاش است تا یک نمایش بزرگ را روی صحنه ببرند و با مشکلات و ماجراهای خنده‌داری روبرو می‌شوند.", "Comedy"),
    ("یک پسر نوجوان که به تازگی به شهر جدیدی نقل مکان کرده است، تلاش می‌کند تا دوستان جدید پیدا کند.", "Drama"),
    ("یک ورزشکار جوان تلاش می‌کند تا به تیم ملی کشورش راه پیدا کند و در این راه با چالش‌های زیادی روبرو می‌شود.", "Drama"),
    ("یک زن و مرد در یک ماجرای عشقی پیچیده قرار می‌گیرند و تلاش می‌کنند تا موانع را پشت سر بگذارند.", "Drama")
]


In [35]:
def predict_genre(text, model, tokenizer):
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    logits = outputs.logits
    predicted_class_idx = logits.argmax(axis=1).item()
    predicted_label = label_encoder.inverse_transform([predicted_class_idx])[0]
    
    return predicted_label


In [36]:
# Predict genres for the examples
true_labels = [genre for _, genre in examples]
predicted_labels = [predict_genre(text, model, tokenizer) for text, _ in examples]

# Evaluate accuracy
correct_predictions = sum([true == pred for true, pred in zip(true_labels, predicted_labels)])
accuracy = correct_predictions / len(examples)

print(f'Predicted genres: {predicted_labels}')
print(f'True genres: {true_labels}')
print(f'Number of correct predictions: {correct_predictions} out of {len(examples)}')
print(f'Accuracy: {accuracy:.2f}')

# Calculate and print additional metrics
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

f1_macro = f1_score(true_labels, predicted_labels, average='macro')
f1_micro = f1_score(true_labels, predicted_labels, average='micro')
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print(f'F1 Score (Macro): {f1_macro:.2f}')
print(f'F1 Score (Micro): {f1_micro:.2f}')
print(f'Precision (Macro): {precision:.2f}')
print(f'Recall (Macro): {recall:.2f}')
print('Confusion Matrix:')
print(conf_matrix)


Predicted genres: ['Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Drama']
True genres: ['Drama', 'Drama', 'Comedy', 'Drama', 'Comedy', 'Drama', 'Comedy', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Drama', 'Comedy', 'Drama', 'Drama', 'Drama']
Number of correct predictions: 28 out of 30
Accuracy: 0.93
F1 Score (Macro): 0.91
F1 Score (Micro): 0.93
Precision (Macro): 0.91
Recall (Macro): 0.91
Confusion Matrix:
[[ 7  1]
 [ 1 21]]
